<a href="https://colab.research.google.com/github/Ash100/Trainings/blob/main/CRISPR_based_sgRNA_Design_HU_June_2025.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 **Fundamentals of Genome Editing and gRNA Design**
## Training Workshop for GEABM-2025, (June 19-20, 2025)-Hazara University
### Author: **Dr. Ashfaq Ahmad** (_Ph.D, Structure Biology, Postdoc Bioinformatics)_

This notebook demonstration provides a practical demonstration of fundamental steps in genome editing, focusing on acquiring and manipulating DNA sequences, and then designing **guide RNAs (gRNAs)** for CRISPR-Cas9 systems.

---

## 📑 Table of Contents
1. [Sequence Acquisition and Manipulation](#sequence-acquisition-and-manipulation)  
    1.1. [Fetching a DNA Sequence from NCBI](#fetching-a-dna-sequence-from-ncbi)  
    1.2. [Basic Sequence Operations](#basic-sequence-operations)  
2. [Guide RNA (gRNA) Design](#guide-rna-grna-design)  
    2.1. [Identifying PAM Sites and Target Sequences](#identifying-pam-sites-and-target-sequences)  
    2.2. [gRNA Structure and Candidates](#grna-structure-and-candidates)  
    2.3. [Simplified Off-Target Consideration](#simplified-off-target-consideration)  
    2.4. [Simplified On-Target Efficiency (Conceptual)](#simplified-on-target-efficiency-conceptual)

In [ ]:
#@title Install and Import necessary programs and libraries
!pip install biopython

# Import necessary libraries
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import re

# Set email for NCBI Entrez (required)
Entrez.email = "your.email@example.com"  # Replace with your email

# 1.1 Fetching a DNA Sequence from NCBI

## Biological Context
The SAL1 gene in *Hordeum vulgare* (barley) encodes a protein involved in salt tolerance, a critical trait for crops in saline soils. SAL1 is a homolog of Arabidopsis SAL1, which regulates stress responses by modulating inositol phosphate signaling. Public databases like NCBI's GenBank store its sequence (accession EU096087.1), enabling bioinformatics analysis for genetic engineering or CRISPR editing. The Entrez API provides programmatic access to GenBank, requiring an email for tracking.

## Objective
Retrieve the SAL1 sequence from NCBI and display its metadata (ID, description, length) to confirm successful acquisition.

In [ ]:
#@title Fetch a DNA sequence from the NCBI
def fetch_sequence(accession):
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
        record = SeqIO.read(handle, "fasta")
        handle.close()
        return record
    except Exception as e:
        print(f"Error fetching sequence: {e}")
        return None

# Example: Fetch BRCA1 gene sequence (example accession)
accession = "EU096087.1"  # SAL1 gene
record = fetch_sequence(accession)

if record:
    print(f"Sequence ID: {record.id}")
    print(f"Description: {record.description}")
    print(f"Sequence length: {len(record.seq)} bp")
    print(f"First 100 bp: {record.seq[:100]}")
else:
    print("Failed to fetch sequence.")

# 1.2 Basic Sequence Operations

## Biological Context
DNA serves as a template for life: transcription converts DNA to RNA (replacing T with U), and translation converts RNA to proteins using the genetic code. The reverse complement represents the opposite DNA strand, as genes can be on either strand. GC content (percentage of G and C bases) affects DNA stability, replication, and mutation rates. High GC content in SAL1's regulatory regions may indicate stable structures.

## Objective
Perform transcription, reverse complementation, and GC content calculation on SAL1.

In [ ]:
#@title Basic sequence operations
if record:
    dna_seq = record.seq
    print(f"Original sequence (first 100 bp): {dna_seq[:100]}")

    # Transcription (DNA to RNA)
    rna_seq = dna_seq.transcribe()
    print(f"RNA sequence (first 100 bp): {rna_seq[:100]}")

    # Translation (DNA to protein)
    protein_seq = dna_seq.translate(to_stop=True)
    print(f"Protein sequence (first 50 aa): {protein_seq[:50]}")

    # Reverse complement
    rev_comp = dna_seq.reverse_complement()
    print(f"Reverse complement (first 100 bp): {rev_comp[:100]}")

    # GC content
    gc_count = dna_seq.count("G") + dna_seq.count("C")
    gc_percent = (gc_count / len(dna_seq)) * 100
    print(f"GC content: {gc_percent:.2f}%")

    # Visualization: Base composition pie chart
    base_counts = {'A': dna_seq.count('A'), 'T': dna_seq.count('T'),
                   'G': dna_seq.count('G'), 'C': dna_seq.count('C')}
    plt.figure(figsize=(6, 6))
    plt.pie(base_counts.values(), labels=base_counts.keys(), colors=['red', 'blue', 'green', 'purple'], autopct='%1.1f%%')
    plt.title('Base Composition of SAL1 Sequence')
    plt.show()
else:
    print("No sequence available for operations.")

# 2.1 Identifying PAM Sites and Target Sequences

## Biological Context
CRISPR/Cas9 requires a protospacer adjacent motif (PAM), typically NGG for SpCas9, to initiate DNA binding. The 20 bp upstream of the PAM (protospacer) pairs with the guide RNA (gRNA). PAM sites dictate where Cas9 can cut, making their identification crucial for CRISPR design. In SAL1, targeting the coding region could disrupt salt tolerance, aiding functional studies or enhancing stress resistance.

## PAM Mechanics
Cas9 recognizes NGG and cleaves DNA ~3–4 bp upstream, triggering repair pathways like non-homologous end joining (NHEJ). PAM sites occur every ~8–12 bp in random DNA, but their density varies. In SAL1's compact sequence, PAM sites are fewer but sufficient for targeted editing.

## Objective
Locate NGG PAM sites in SAL1 and extract 20 bp target sequences for gRNA design.

In [ ]:
#@title Identify PAM sites (NGG for SpCas9) and target sequences
def find_pam_sites(sequence, pam="NGG"):
    pam_pattern = pam.replace("N", "[ATCG]").replace("G", "G")
    pam_regex = re.compile(pam_pattern)
    pam_sites = []

    for match in pam_regex.finditer(str(sequence)):
        start = match.start()
        # Target sequence is 20 bp upstream of PAM
        if start >= 20:
            target_seq = sequence[start-20:start]
            pam_sites.append((start, target_seq, match.group()))

    return pam_sites

# Find PAM sites in the sequence
if record:
    pam_sites = find_pam_sites(record.seq)
    print(f"Found {len(pam_sites)} PAM sites (NGG).")
    print("First 5 PAM sites (position, target sequence, PAM):")
    for i, (pos, target, pam) in enumerate(pam_sites[:5]):
        print(f"Site {i+1}: Position {pos}, Target: {target}, PAM: {pam}")

    # Visualization: PAM site distribution histogram
    positions = [p[0] for p in pam_sites]
    plt.figure(figsize=(10, 5))
    plt.hist(positions, bins=50, color='purple', alpha=0.7)
    plt.xlabel('Position in Sequence (bp)')
    plt.ylabel('Number of PAM Sites')
    plt.title('Distribution of NGG PAM Sites in SAL1')
    plt.show()
else:
    print("No sequence available for PAM site analysis.")

# 2.2 gRNA Structure and Candidates

## Biological Context
Guide RNA (gRNA) directs Cas9 to the target DNA via a 20 bp spacer that base-pairs with the protospacer. The gRNA also includes a scaffold (~80 nt) that binds Cas9, enabling DNA cleavage. In SAL1, gRNAs can knock out the gene to study salt tolerance or introduce mutations to enhance it, supporting barley's adaptation to saline environments.

## gRNA Structure
The spacer ensures specificity, while the scaffold (tracrRNA-derived) forms RNA secondary structures critical for Cas9 activation. The scaffold is conserved across SpCas9 applications, ensuring reliable function in barley.

## Objective
Generate gRNA sequences for top PAM sites in SAL1 by combining 20 bp spacers with a standard scaffold.

In [ ]:
#@title Generate gRNA candidates with basic structure
def generate_grna(target_seq):
    grna = Seq(target_seq)
    # Basic gRNA structure: 20 bp target + scaffold (simplified)
    scaffold = "GUUUUAGAGCUAGAAAUAGCAAGUUAAAAUAAGGCUAGUCCGUUAUCAACUUGAAAAAGUGGCACCGAGUCGGUGCUUUU"
    return str(grna) + scaffold

# Select top 5 PAM sites and generate gRNA
if record:
    pam_sites = find_pam_sites(record.seq)
    print("Top 5 gRNA candidates (20 bp target + scaffold):")
    for i, (pos, target, pam) in enumerate(pam_sites[:5]):
        grna_seq = generate_grna(target)
        print(f"gRNA {i+1} (Position {pos}):")
        print(f"Target: {target}")
        print(f"PAM: {pam}")
        print(f"gRNA sequence (first 50 bp + scaffold): {grna_seq[:50]}...")

    # Visualization: Spacer length validation bar plot
    spacers = [len(str(p[1])) for p in pam_sites[:5]]
    plt.figure(figsize=(6, 4))
    plt.bar(range(1, 6), spacers, color='orange')
    plt.axhline(y=20, color='green', linestyle='--', label='Expected Length (20 bp)')
    plt.xlabel('gRNA Candidate #')
    plt.ylabel('Spacer Length (bp)')
    plt.title('Spacer Lengths of Top 5 gRNA Candidates')
    plt.legend()
    plt.show()
else:
    print("No sequence available for gRNA design.")

# 2.3 Simplified Off-Target Consideration

## Biological Context
Off-target effects occur when Cas9 cleaves unintended DNA due to partial gRNA matches, potentially disrupting unrelated genes. In SAL1 editing, off-targets in other stress-related genes could affect barley's resilience. Off-target sites typically have 1–3 mismatches in the spacer, especially distal from the PAM, reducing Cas9 specificity.

## Off-Target Biology
Cas9 tolerates mismatches, particularly in the 5' end of the spacer. Off-target analysis scans genomes for similar sequences, but here we simplify by searching SAL1 itself. In barley, off-targets in stress pathways could complicate salt tolerance studies.

## Objective
Identify potential off-target sites for the first SAL1 gRNA by counting mismatches within the SAL1 sequence.

In [ ]:
#@title Simplified off-target scoring (basic mismatch counting)
def check_off_target(target_seq, genome_seq, max_mismatches=3):
    target = str(target_seq)
    off_targets = []
    for i in range(len(genome_seq) - len(target)):
        sub_seq = str(genome_seq[i:i+len(target)])
        mismatches = sum(a != b for a, b in zip(target, sub_seq))
        if mismatches <= max_mismatches:
            off_targets.append((i, sub_seq, mismatches))
    return off_targets

# Check off-targets for the first gRNA candidate
if record and pam_sites:
    target_seq = pam_sites[0][1]  # First target sequence
    off_targets = check_off_target(target_seq, record.seq)
    print(f"Off-target analysis for target: {target_seq}")
    print(f"Found {len(off_targets)} potential off-target sites with <= 3 mismatches:")
    for i, (pos, seq, mismatches) in enumerate(off_targets[:5]):
        print(f"Off-target {i+1}: Position {pos}, Sequence: {seq}, Mismatches: {mismatches}")

    # Visualization: Mismatch distribution histogram
    mismatch_counts = [t[2] for t in off_targets]
    plt.figure(figsize=(6, 4))
    plt.hist(mismatch_counts, bins=range(4), align='left', color='red', alpha=0.7)
    plt.xlabel('Number of Mismatches')
    plt.ylabel('Number of Off-Target Sites')
    plt.title('Mismatch Distribution for First SAL1 gRNA')
    plt.show()
else:
    print("No sequence or PAM sites available for off-target analysis.")

# 2.4 Simplified On-Target Efficiency (Conceptual)

## Biological Context
gRNA efficiency determines Cas9's ability to cut the target DNA. Efficiency depends on GC content (30–70% is optimal), sequence composition, and chromatin state. In SAL1, efficient gRNAs ensure precise edits to study or enhance salt tolerance, critical for barley in saline soils.

## Efficiency Factors
- **GC Content**: Moderate GC improves Cas9 binding without forming stable RNA structures.
- **3' End**: Avoiding GG at the spacer's end enhances efficiency.
- **Complexity**: Diverse sequences reduce off-target risk and improve specificity.

## Objective
Score the top 5 SAL1 gRNA candidates based on simplified efficiency criteria.

In [ ]:
#@title Simplified on-target efficiency scoring (GC content and basic rules)
def score_grna_efficiency(target_seq):
    gc_count = target_seq.count("G") + target_seq.count("C")
    gc_percent = (gc_count / len(target_seq)) * 100

    # Simplified rules for efficiency
    score = 0
    if 30 <= gc_percent <= 70:  # Optimal GC content
        score += 50
    if not target_seq.endswith("GG"):  # Avoid GG at 3' end
        score += 30
    if len(set(target_seq)) > 10:  # Sequence complexity
        score += 20

    return score, gc_percent

# Score top 5 gRNA candidates
if record and pam_sites:
    print("On-target efficiency scores for top 5 gRNA candidates:")
    scores = []
    for i, (pos, target, pam) in enumerate(pam_sites[:5]):
        score, gc = score_grna_efficiency(target)
        scores.append(score)
        print(f"gRNA {i+1} (Position {pos}):")
        print(f"Target: {target}")
        print(f"GC content: {gc:.2f}%")
        print(f"Efficiency score: {score}/100")

    # Visualization: Efficiency scores bar plot
    plt.figure(figsize=(6, 4))
    plt.bar(range(1, 6), scores, color='green')
    plt.xlabel('gRNA Candidate #')
    plt.ylabel('Efficiency Score (out of 100)')
    plt.title('Efficiency Scores of Top 5 SAL1 gRNA Candidates')
    plt.show()
else:
    print("No sequence or PAM sites available for efficiency scoring.")

### Thank You So Much - I hope you learned Something New